In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("divorce.csv", low_memory=False)
df.head()

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
0,2,2,4,1,0,0,0,0,0,0,...,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,...,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,...,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,...,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,...,2,1,2,3,2,2,2,1,0,1


## Differences in Means (Divorced vs. Married)

In [3]:
# Create masks for divorced and married respondents
divorced = df["Class"] == 1 # 1 means divorced
married = df["Class"] == 0 # 0 means stayed divorced

# 0 means agree, 4 means disagree
print(df[divorced]["Atr1"].mean())
print(df[married]["Atr1"].mean())

3.1904761904761907
0.3953488372093023


In [4]:
def divorced_vs_not(question_num):
    """
    Calculates the average response of divorced respondents minus the
    average response of married respondents to a given question
    
    question_num: the integer number of the question
    """
    column_name = "Atr" + str(question_num)
    divorced_avg = df[divorced][column_name].mean()
    married_avg = df[married][column_name].mean()
    return divorced_avg - married_avg

In [5]:
# Find the most 10 questions with the highest differences
pairs = [(i, divorced_vs_not(i)) for i in range(1, 55)]
pairs.sort(key=lambda x: x[1], reverse=True)

for pair in pairs[:10]:
    print(pair)

(40, 3.3621262458471763)
(36, 3.179401993355482)
(35, 3.169158361018826)
(39, 3.0730897009966776)
(41, 3.070874861572536)
(33, 3.0664451827242525)
(38, 3.055924695459579)
(19, 3.0390365448504983)
(11, 3.0166112956810633)
(17, 2.992248062015504)


## Predicting: Neural Network

In [6]:
df.shape

(170, 55)

In [7]:
# Normalize data
for i in range(1, 55):
    df["Atr"+str(i)] /= 4
df.head()

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
0,0.50,0.5,1.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00,...,0.50,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,1
1,1.00,1.0,1.00,1.00,1.00,0.00,0.00,1.00,1.00,1.00,...,0.50,0.50,0.75,1.00,1.00,1.00,1.00,0.50,0.50,1
2,0.50,0.5,0.50,0.50,0.25,0.75,0.50,0.25,0.25,0.50,...,0.75,0.50,0.75,0.25,0.25,0.25,0.50,0.50,0.50,1
3,0.75,0.5,0.75,0.50,0.75,0.75,0.75,0.75,0.75,0.75,...,0.50,0.50,0.75,0.75,0.75,0.75,0.50,0.50,0.50,1
4,0.50,0.5,0.25,0.25,0.25,0.25,0.00,0.00,0.00,0.00,...,0.50,0.25,0.50,0.75,0.50,0.50,0.50,0.25,0.00,1


In [8]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Get training data
msk = np.random.rand(len(df)) < 0.2
train = df[msk]
test = df[~msk]

# Slice attributes and labels
train_data = train.iloc[:,:54]
train_labels = train.iloc[:,54:]

test_data = test.iloc[:,:54]
test_labels = test.iloc[:,54:]

# Build the model.
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=54))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
model.fit(train_data, train_labels, epochs=1000, batch_size=32)

Using TensorFlow backend.


Epoch 1/1000
27/27 [==============================] - 0s 4ms/step - loss: 0.6884 - accuracy: 0.4815
Epoch 2/1000
27/27 [==============================] - 0s 51us/step - loss: 0.6433 - accuracy: 0.7407
Epoch 3/1000
27/27 [==============================] - 0s 49us/step - loss: 0.5895 - accuracy: 0.7407
Epoch 4/1000
27/27 [==============================] - 0s 41us/step - loss: 0.5443 - accuracy: 0.7037
Epoch 5/1000
27/27 [==============================] - 0s 41us/step - loss: 0.5110 - accuracy: 0.7037
Epoch 6/1000
27/27 [==============================] - 0s 38us/step - loss: 0.4854 - accuracy: 0.7037
Epoch 7/1000
27/27 [==============================] - 0s 39us/step - loss: 0.4656 - accuracy: 0.7037
Epoch 8/1000
27/27 [==============================] - 0s 53us/step - loss: 0.4493 - accuracy: 0.7037
Epoch 9/1000
27/27 [==============================] - 0s 46us/step - loss: 0.4356 - accuracy: 0.7037
Epoch 10/1000
27/27 [==============================] - 0s 52us/step - loss: 0.4244 - accurac

Epoch 82/1000
27/27 [==============================] - 0s 57us/step - loss: 0.1495 - accuracy: 1.0000
Epoch 83/1000
27/27 [==============================] - 0s 44us/step - loss: 0.1470 - accuracy: 1.0000
Epoch 84/1000
27/27 [==============================] - 0s 42us/step - loss: 0.1445 - accuracy: 1.0000
Epoch 85/1000
27/27 [==============================] - 0s 36us/step - loss: 0.1421 - accuracy: 1.0000
Epoch 86/1000
27/27 [==============================] - 0s 46us/step - loss: 0.1396 - accuracy: 1.0000
Epoch 87/1000
27/27 [==============================] - 0s 49us/step - loss: 0.1373 - accuracy: 1.0000
Epoch 88/1000
27/27 [==============================] - 0s 47us/step - loss: 0.1350 - accuracy: 1.0000
Epoch 89/1000
27/27 [==============================] - 0s 43us/step - loss: 0.1326 - accuracy: 1.0000
Epoch 90/1000
27/27 [==============================] - 0s 40us/step - loss: 0.1303 - accuracy: 1.0000
Epoch 91/1000
27/27 [==============================] - 0s 45us/step - loss: 0.1280

Epoch 162/1000
27/27 [==============================] - 0s 41us/step - loss: 0.0264 - accuracy: 1.0000
Epoch 163/1000
27/27 [==============================] - 0s 43us/step - loss: 0.0257 - accuracy: 1.0000
Epoch 164/1000
27/27 [==============================] - 0s 52us/step - loss: 0.0251 - accuracy: 1.0000
Epoch 165/1000
27/27 [==============================] - 0s 37us/step - loss: 0.0245 - accuracy: 1.0000
Epoch 166/1000
27/27 [==============================] - 0s 35us/step - loss: 0.0239 - accuracy: 1.0000
Epoch 167/1000
27/27 [==============================] - 0s 51us/step - loss: 0.0232 - accuracy: 1.0000
Epoch 168/1000
27/27 [==============================] - 0s 39us/step - loss: 0.0226 - accuracy: 1.0000
Epoch 169/1000
27/27 [==============================] - 0s 46us/step - loss: 0.0220 - accuracy: 1.0000
Epoch 170/1000
27/27 [==============================] - 0s 36us/step - loss: 0.0215 - accuracy: 1.0000
Epoch 171/1000
27/27 [==============================] - 0s 38us/step - lo

Epoch 242/1000
27/27 [==============================] - 0s 52us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 243/1000
27/27 [==============================] - 0s 85us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 244/1000
27/27 [==============================] - 0s 51us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 245/1000
27/27 [==============================] - 0s 36us/step - loss: 0.0021 - accuracy: 1.0000
Epoch 246/1000
27/27 [==============================] - 0s 60us/step - loss: 0.0021 - accuracy: 1.0000
Epoch 247/1000
27/27 [==============================] - 0s 49us/step - loss: 0.0020 - accuracy: 1.0000
Epoch 248/1000
27/27 [==============================] - 0s 50us/step - loss: 0.0019 - accuracy: 1.0000
Epoch 249/1000
27/27 [==============================] - 0s 43us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 250/1000
27/27 [==============================] - 0s 61us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 251/1000
27/27 [==============================] - 0s 55us/step - lo

...

Epoch 628/1000
27/27 [==============================] - 0s 41us/step - loss: 1.1072e-08 - accuracy: 1.0000
Epoch 629/1000
27/27 [==============================] - 0s 49us/step - loss: 1.0961e-08 - accuracy: 1.0000
Epoch 630/1000
27/27 [==============================] - 0s 39us/step - loss: 1.0872e-08 - accuracy: 1.0000
Epoch 631/1000
27/27 [==============================] - 0s 38us/step - loss: 1.0765e-08 - accuracy: 1.0000
Epoch 632/1000
27/27 [==============================] - 0s 50us/step - loss: 1.0679e-08 - accuracy: 1.0000
Epoch 633/1000
27/27 [==============================] - 0s 42us/step - loss: 1.0596e-08 - accuracy: 1.0000
Epoch 634/1000
27/27 [==============================] - 0s 39us/step - loss: 1.0492e-08 - accuracy: 1.0000
Epoch 635/1000
27/27 [==============================] - 0s 58us/step - loss: 1.0412e-08 - accuracy: 1.0000
Epoch 636/1000
27/27 [==============================] - 0s 51us/step - loss: 1.0335e-08 - accuracy: 1.0000
Epoch 637/1000
27/27 [===============

Epoch 705/1000
27/27 [==============================] - 0s 38us/step - loss: 6.7156e-09 - accuracy: 1.0000
Epoch 706/1000
27/27 [==============================] - 0s 45us/step - loss: 6.6949e-09 - accuracy: 1.0000
Epoch 707/1000
27/27 [==============================] - 0s 45us/step - loss: 6.6239e-09 - accuracy: 1.0000
Epoch 708/1000
27/27 [==============================] - 0s 35us/step - loss: 6.6031e-09 - accuracy: 1.0000
Epoch 709/1000
27/27 [==============================] - 0s 37us/step - loss: 6.5830e-09 - accuracy: 1.0000
Epoch 710/1000
27/27 [==============================] - 0s 52us/step - loss: 6.5635e-09 - accuracy: 1.0000
Epoch 711/1000
27/27 [==============================] - 0s 40us/step - loss: 6.5446e-09 - accuracy: 1.0000
Epoch 712/1000
27/27 [==============================] - 0s 36us/step - loss: 6.4741e-09 - accuracy: 1.0000
Epoch 713/1000
27/27 [==============================] - 0s 39us/step - loss: 6.4554e-09 - accuracy: 1.0000
Epoch 714/1000
27/27 [===============

Epoch 782/1000
27/27 [==============================] - 0s 42us/step - loss: 5.0610e-09 - accuracy: 1.0000
Epoch 783/1000
27/27 [==============================] - 0s 45us/step - loss: 5.0564e-09 - accuracy: 1.0000
Epoch 784/1000
27/27 [==============================] - 0s 47us/step - loss: 4.9825e-09 - accuracy: 1.0000
Epoch 785/1000
27/27 [==============================] - 0s 41us/step - loss: 4.9777e-09 - accuracy: 1.0000
Epoch 786/1000
27/27 [==============================] - 0s 40us/step - loss: 4.9733e-09 - accuracy: 1.0000
Epoch 787/1000
27/27 [==============================] - 0s 40us/step - loss: 4.9689e-09 - accuracy: 1.0000
Epoch 788/1000
27/27 [==============================] - 0s 54us/step - loss: 4.9646e-09 - accuracy: 1.0000
Epoch 789/1000
27/27 [==============================] - 0s 47us/step - loss: 4.9603e-09 - accuracy: 1.0000
Epoch 790/1000
27/27 [==============================] - 0s 37us/step - loss: 4.9563e-09 - accuracy: 1.0000
Epoch 791/1000
27/27 [===============

Epoch 859/1000
27/27 [==============================] - 0s 52us/step - loss: 4.1672e-09 - accuracy: 1.0000
Epoch 860/1000
27/27 [==============================] - 0s 37us/step - loss: 4.1670e-09 - accuracy: 1.0000
Epoch 861/1000
27/27 [==============================] - 0s 42us/step - loss: 4.1670e-09 - accuracy: 1.0000
Epoch 862/1000
27/27 [==============================] - 0s 49us/step - loss: 4.1668e-09 - accuracy: 1.0000
Epoch 863/1000
27/27 [==============================] - 0s 49us/step - loss: 4.1667e-09 - accuracy: 1.0000
Epoch 864/1000
27/27 [==============================] - 0s 51us/step - loss: 4.1666e-09 - accuracy: 1.0000
Epoch 865/1000
27/27 [==============================] - 0s 42us/step - loss: 4.1667e-09 - accuracy: 1.0000
Epoch 866/1000
27/27 [==============================] - 0s 54us/step - loss: 4.1667e-09 - accuracy: 1.0000
Epoch 867/1000
27/27 [==============================] - 0s 40us/step - loss: 4.1668e-09 - accuracy: 1.0000
Epoch 868/1000
27/27 [===============

Epoch 936/1000
27/27 [==============================] - 0s 56us/step - loss: 3.6700e-09 - accuracy: 1.0000
Epoch 937/1000
27/27 [==============================] - 0s 43us/step - loss: 3.6716e-09 - accuracy: 1.0000
Epoch 938/1000
27/27 [==============================] - 0s 48us/step - loss: 3.6731e-09 - accuracy: 1.0000
Epoch 939/1000
27/27 [==============================] - 0s 36us/step - loss: 3.6747e-09 - accuracy: 1.0000
Epoch 940/1000
27/27 [==============================] - 0s 42us/step - loss: 3.6763e-09 - accuracy: 1.0000
Epoch 941/1000
27/27 [==============================] - 0s 40us/step - loss: 3.6779e-09 - accuracy: 1.0000
Epoch 942/1000
27/27 [==============================] - 0s 44us/step - loss: 3.6796e-09 - accuracy: 1.0000
Epoch 943/1000
27/27 [==============================] - 0s 53us/step - loss: 3.6813e-09 - accuracy: 1.0000
Epoch 944/1000
27/27 [==============================] - 0s 36us/step - loss: 3.6829e-09 - accuracy: 1.0000
Epoch 945/1000
27/27 [===============

In [9]:
model.evaluate(test_data, test_labels, batch_size=32)

143/143 [==============================] - 0s 158us/step


[0.1326318968468431, 0.9790209531784058]

# Categorizing Questions

In [10]:
categories = pd.read_csv("divorce_values.csv")
categories.tail()

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Unnamed: 54
0,Communications,Communications,Communications,Communications,Experiences,Experiences,Experiences,Experiences,Experiences,Values,...,Acquiesce,Acquiesce,Self-righteous,Self-righteous,Self-righteous,Self-righteous,Respect,Respect,Respect,Zero
1,Positive,Positive,Positive,Positive,Positive,Negative,Negative,Positive,Positive,Positive,...,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Zero


In [11]:
for i in range(1, 55):
    #print(divorce_categories["Atr" + str(i)].shape)
    if categories["Atr" + str(i)].iloc[1] == 'Negative':
        #print(4 - divorce_categories["Atr" + str(i)])
        df["Atr" + str(i)] = 1 - df["Atr" + str(i)]

In [12]:
unique_categories = categories.iloc[0,:54].unique()
category_avgs = []
for row in range(len(df)):
    avgs = []
    for category in unique_categories:
        values = []
        for col in range(54):
            if categories.iloc[0, col] == category:
                values.append(df.iloc[row,col])
        avgs.append(sum(values) / len(values))
    avgs.append(df.iloc[row,54])
    category_avgs.append(avgs)

headers = list(unique_categories)
headers.append("Divorce Status")
avg_df = pd.DataFrame(category_avgs, columns=headers)
export_csv = avg_df.to_csv(r'./output.csv', index=None, header=True)
avg_df.head()

,Communications,Experiences,Values,Empathy,Respect,Acquiesce,Self-righteous,Divorce Status
0,0.50000,0.321429,0.100,0.027778,0.625,0.583333,0.3125,1
1,0.62500,0.964286,0.750,0.277778,0.450,0.375000,0.0625,1
2,0.43750,0.464286,0.625,0.444444,0.550,0.375000,0.6250,1
3,0.40625,0.642857,0.750,0.333333,0.500,0.416667,0.2500,1
4,0.59375,0.321429,0.225,0.194444,0.825,0.583333,0.4375,1


In [13]:
divorced_mask = avg_df["Divorce Status"] == 1 # 1 means divorced
married_mask = avg_df["Divorce Status"] == 0 # 0 means stayed divorced

for category in unique_categories:
    divorced_avg = avg_df[divorced_mask][category].mean()
    married_avg = avg_df[married_mask][category].mean()
    print(category)
    print(abs(divorced_avg - married_avg), divorced_avg, married_avg)

Communications
0.08209613787375408 0.42299107142857145 0.5050872093023255
Experiences
0.4374999999999999 0.7410714285714285 0.3035714285714286
Values
0.6760866555924697 0.7342261904761906 0.05813953488372092
Empathy
0.6468100159960625 0.6878306878306878 0.041020671834625315
Respect
0.6777547065337763 0.15654761904761902 0.8343023255813954
Acquiesce
0.48397702104097473 0.16815476190476195 0.6521317829457367
Self-righteous
0.46849044850498334 0.13616071428571427 0.6046511627906976


# Train the network based on category averages

In [14]:
# Get training data
msk = np.random.rand(len(avg_df)) < 0.8
train = avg_df[msk]
test = avg_df[~msk]

# Slice attributes and labels
train_data = train.iloc[:,:7]
train_labels = train.iloc[:,7:]

print(train_data.shape)
print(train_labels.shape)

test_data = test.iloc[:,:7]
test_labels = test.iloc[:,7:]

# Build the model.
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=7))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
model.fit(train_data, train_labels, epochs=1000, batch_size=32)

(134, 7)
(134, 1)
Epoch 1/1000
134/134 [==============================] - 0s 702us/step - loss: 0.6753 - accuracy: 0.4776
Epoch 2/1000
134/134 [==============================] - 0s 51us/step - loss: 0.6130 - accuracy: 0.6642
Epoch 3/1000
134/134 [==============================] - 0s 52us/step - loss: 0.5702 - accuracy: 0.8433
Epoch 4/1000
134/134 [==============================] - 0s 52us/step - loss: 0.5335 - accuracy: 0.8731
Epoch 5/1000
134/134 [==============================] - 0s 74us/step - loss: 0.4992 - accuracy: 0.8955
Epoch 6/1000
134/134 [==============================] - 0s 75us/step - loss: 0.4732 - accuracy: 0.9403
Epoch 7/1000
134/134 [==============================] - 0s 67us/step - loss: 0.4498 - accuracy: 0.9478
Epoch 8/1000
134/134 [==============================] - 0s 52us/step - loss: 0.4271 - accuracy: 0.9552
Epoch 9/1000
134/134 [==============================] - 0s 49us/step - loss: 0.4073 - accuracy: 0.9627
Epoch 10/1000
134/134 [==============================]

134/134 [==============================] - 0s 44us/step - loss: 0.0465 - accuracy: 0.9851
Epoch 80/1000
134/134 [==============================] - 0s 55us/step - loss: 0.0459 - accuracy: 0.9851
Epoch 81/1000
134/134 [==============================] - 0s 39us/step - loss: 0.0459 - accuracy: 0.9851
Epoch 82/1000
134/134 [==============================] - 0s 50us/step - loss: 0.0450 - accuracy: 0.9851
Epoch 83/1000
134/134 [==============================] - 0s 46us/step - loss: 0.0447 - accuracy: 0.9851
Epoch 84/1000
134/134 [==============================] - 0s 47us/step - loss: 0.0448 - accuracy: 0.9851
Epoch 85/1000
134/134 [==============================] - 0s 42us/step - loss: 0.0443 - accuracy: 0.9851
Epoch 86/1000
134/134 [==============================] - 0s 50us/step - loss: 0.0453 - accuracy: 0.9851
Epoch 87/1000
134/134 [==============================] - 0s 87us/step - loss: 0.0440 - accuracy: 0.9851
Epoch 88/1000
134/134 [==============================] - 0s 59us/step - loss: 

134/134 [==============================] - 0s 51us/step - loss: 0.0302 - accuracy: 0.9851
Epoch 158/1000
134/134 [==============================] - 0s 43us/step - loss: 0.0293 - accuracy: 0.9851
Epoch 159/1000
134/134 [==============================] - 0s 41us/step - loss: 0.0301 - accuracy: 0.9851
Epoch 160/1000
134/134 [==============================] - 0s 34us/step - loss: 0.0291 - accuracy: 0.9851
Epoch 161/1000
134/134 [==============================] - 0s 43us/step - loss: 0.0286 - accuracy: 0.9851
Epoch 162/1000
134/134 [==============================] - 0s 37us/step - loss: 0.0301 - accuracy: 0.9851
Epoch 163/1000
134/134 [==============================] - 0s 40us/step - loss: 0.0295 - accuracy: 0.9851
Epoch 164/1000
134/134 [==============================] - 0s 32us/step - loss: 0.0292 - accuracy: 0.9851
Epoch 165/1000
134/134 [==============================] - 0s 38us/step - loss: 0.0283 - accuracy: 0.9851
Epoch 166/1000
134/134 [==============================] - 0s 41us/step

134/134 [==============================] - 0s 37us/step - loss: 0.0184 - accuracy: 0.9925
Epoch 236/1000
134/134 [==============================] - 0s 38us/step - loss: 0.0179 - accuracy: 0.9925
Epoch 237/1000
134/134 [==============================] - 0s 47us/step - loss: 0.0174 - accuracy: 0.9925
Epoch 238/1000
134/134 [==============================] - 0s 34us/step - loss: 0.0169 - accuracy: 0.9925
Epoch 239/1000
134/134 [==============================] - 0s 39us/step - loss: 0.0175 - accuracy: 0.9925
Epoch 240/1000
134/134 [==============================] - 0s 36us/step - loss: 0.0170 - accuracy: 0.9925
Epoch 241/1000
134/134 [==============================] - 0s 42us/step - loss: 0.0174 - accuracy: 0.9925
Epoch 242/1000
134/134 [==============================] - 0s 39us/step - loss: 0.0168 - accuracy: 0.9925
Epoch 243/1000
134/134 [==============================] - 0s 34us/step - loss: 0.0165 - accuracy: 0.9925
Epoch 244/1000
134/134 [==============================] - 0s 47us/step

134/134 [==============================] - 0s 49us/step - loss: 0.0114 - accuracy: 1.0000
Epoch 314/1000
134/134 [==============================] - 0s 34us/step - loss: 0.0111 - accuracy: 1.0000
Epoch 315/1000
134/134 [==============================] - 0s 42us/step - loss: 0.0109 - accuracy: 1.0000
Epoch 316/1000
134/134 [==============================] - 0s 39us/step - loss: 0.0104 - accuracy: 1.0000
Epoch 317/1000
134/134 [==============================] - 0s 40us/step - loss: 0.0105 - accuracy: 0.9925
Epoch 318/1000
134/134 [==============================] - 0s 36us/step - loss: 0.0107 - accuracy: 0.9925
Epoch 319/1000
134/134 [==============================] - 0s 33us/step - loss: 0.0103 - accuracy: 0.9925
Epoch 320/1000
134/134 [==============================] - 0s 40us/step - loss: 0.0115 - accuracy: 0.9925
Epoch 321/1000
134/134 [==============================] - 0s 37us/step - loss: 0.0101 - accuracy: 1.0000
Epoch 322/1000
134/134 [==============================] - 0s 36us/step

...

134/134 [==============================] - 0s 39us/step - loss: 3.3661e-04 - accuracy: 1.0000
Epoch 700/1000
134/134 [==============================] - 0s 37us/step - loss: 4.0622e-04 - accuracy: 1.0000
Epoch 701/1000
134/134 [==============================] - 0s 32us/step - loss: 3.4354e-04 - accuracy: 1.0000
Epoch 702/1000
134/134 [==============================] - 0s 44us/step - loss: 3.1600e-04 - accuracy: 1.0000
Epoch 703/1000
134/134 [==============================] - 0s 35us/step - loss: 3.1324e-04 - accuracy: 1.0000
Epoch 704/1000
134/134 [==============================] - 0s 39us/step - loss: 3.6207e-04 - accuracy: 1.0000
Epoch 705/1000
134/134 [==============================] - 0s 32us/step - loss: 3.6463e-04 - accuracy: 1.0000
Epoch 706/1000
134/134 [==============================] - 0s 33us/step - loss: 3.2478e-04 - accuracy: 1.0000
Epoch 707/1000
134/134 [==============================] - 0s 45us/step - loss: 2.8235e-04 - accuracy: 1.0000
Epoch 708/1000
134/134 [==========

134/134 [==============================] - 0s 34us/step - loss: 1.4596e-04 - accuracy: 1.0000
Epoch 775/1000
134/134 [==============================] - 0s 42us/step - loss: 1.3362e-04 - accuracy: 1.0000
Epoch 776/1000
134/134 [==============================] - 0s 48us/step - loss: 1.6132e-04 - accuracy: 1.0000
Epoch 777/1000
134/134 [==============================] - 0s 40us/step - loss: 1.2096e-04 - accuracy: 1.0000
Epoch 778/1000
134/134 [==============================] - 0s 49us/step - loss: 1.2727e-04 - accuracy: 1.0000
Epoch 779/1000
134/134 [==============================] - 0s 34us/step - loss: 1.4003e-04 - accuracy: 1.0000
Epoch 780/1000
134/134 [==============================] - 0s 41us/step - loss: 1.1843e-04 - accuracy: 1.0000
Epoch 781/1000
134/134 [==============================] - 0s 36us/step - loss: 1.2823e-04 - accuracy: 1.0000
Epoch 782/1000
134/134 [==============================] - 0s 34us/step - loss: 1.3740e-04 - accuracy: 1.0000
Epoch 783/1000
134/134 [==========

134/134 [==============================] - 0s 37us/step - loss: 4.9535e-05 - accuracy: 1.0000
Epoch 850/1000
134/134 [==============================] - 0s 44us/step - loss: 4.5201e-05 - accuracy: 1.0000
Epoch 851/1000
134/134 [==============================] - 0s 34us/step - loss: 3.8160e-05 - accuracy: 1.0000
Epoch 852/1000
134/134 [==============================] - 0s 35us/step - loss: 3.9577e-05 - accuracy: 1.0000
Epoch 853/1000
134/134 [==============================] - 0s 35us/step - loss: 3.7092e-05 - accuracy: 1.0000
Epoch 854/1000
134/134 [==============================] - 0s 42us/step - loss: 4.1670e-05 - accuracy: 1.0000
Epoch 855/1000
134/134 [==============================] - 0s 34us/step - loss: 4.3958e-05 - accuracy: 1.0000
Epoch 856/1000
134/134 [==============================] - 0s 40us/step - loss: 3.5003e-05 - accuracy: 1.0000
Epoch 857/1000
134/134 [==============================] - 0s 41us/step - loss: 3.8414e-05 - accuracy: 1.0000
Epoch 858/1000
134/134 [==========

134/134 [==============================] - 0s 33us/step - loss: 1.4495e-05 - accuracy: 1.0000
Epoch 925/1000
134/134 [==============================] - 0s 37us/step - loss: 1.2530e-05 - accuracy: 1.0000
Epoch 926/1000
134/134 [==============================] - 0s 43us/step - loss: 1.1259e-05 - accuracy: 1.0000
Epoch 927/1000
134/134 [==============================] - 0s 31us/step - loss: 1.1609e-05 - accuracy: 1.0000
Epoch 928/1000
134/134 [==============================] - 0s 45us/step - loss: 1.2266e-05 - accuracy: 1.0000
Epoch 929/1000
134/134 [==============================] - 0s 36us/step - loss: 1.4364e-05 - accuracy: 1.0000
Epoch 930/1000
134/134 [==============================] - 0s 42us/step - loss: 1.3868e-05 - accuracy: 1.0000
Epoch 931/1000
134/134 [==============================] - 0s 32us/step - loss: 9.8319e-06 - accuracy: 1.0000
Epoch 932/1000
134/134 [==============================] - 0s 33us/step - loss: 1.4459e-05 - accuracy: 1.0000
Epoch 933/1000
134/134 [==========

134/134 [==============================] - 0s 35us/step - loss: 2.7769e-06 - accuracy: 1.0000
Epoch 1000/1000
134/134 [==============================] - 0s 31us/step - loss: 3.0804e-06 - accuracy: 1.0000


In [15]:
model.evaluate(test_data, test_labels, batch_size=32)

36/36 [==============================] - 0s 565us/step


[0.38908431355957873, 0.9722222089767456]